In [394]:
from langchain.tools.retriever import create_retriever_tool
from utils.dependency_setup import get_embeddings, get_vector_store

embeddings = get_embeddings()
vector_store = get_vector_store(embeddings, "langstuffindex")

retriever = vector_store.as_retriever()

In [395]:
from langchain_openai import ChatOpenAI

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4o-mini")

# AGENT state creation

In [433]:
from pydantic import BaseModel, Field
from typing import List, Annotated, Tuple
from langgraph.graph import  MessagesState
import operator
class NodeBuilderState(MessagesState):
    """State for the node builder."""
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str
    next: str = Field(description="Next node to go to")
    schema_info: str = Field(description="Schema information about the node")
    input_schema: str = Field(description="Input schema of the node")
    output_schema: str = Field(description="Output schema of the node")
    description: str = Field(description="Description of the node")
    function_name: str = Field(description="Function name of the node")
    code: str = Field(description="Code for the node")
    toolset: list[str] = Field(description="List of tools to be used in the node by the llm")
    node_type : str = Field(description="Type of node, deterministic if the function is deterministic and requires simple python code generation, ai if the function is not deterministic and requires a llm usage for meeting the requirements")
    node_info: str = Field(description="node information")
    task: str = Field(description="current task")
    final_code: str = Field(description="the final code to output")
    pass


# NODE type identification and deterministic code gen

In [434]:
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal
from langchain_core.messages import HumanMessage


class NodeType(BaseModel):
    node_type: Literal["deterministic", "planner"] = Field(description=
                                                      """Type of node, 
                                                      : deterministic if the function is deterministic and requires simple python code generation,
                                                      : planner if the function is not deterministic, like analysis of input, plan generation or any other thing which is fuzzy logic requires Artificial intelligence for meeting the requirements""")


def determine_node_type(state: NodeBuilderState):
    """Determine the type of node."""
    node_type: str = state["node_type"]
    if node_type == "deterministic":
        return "deterministic_code"
    elif node_type == "planner":
        return "planner"

node_info_prompt= """
You are provided with the following information about the node:
<SchemaInfo>
{schema_info}
</SchemaInfo>
<InputSchema>
{input_schema}
</InputSchema>
<OutputSchema>
{output_schema}
</OutputSchema>
<Description>
{description}
</Description>
<FunctionName>
{function_name}
</FunctionName>

Below is the skeleton of the function that you need to implement:
def {function_name}(state:{input_schema}) -> {output_schema}:
    \"\"\"{description}\"\"\"
    # Implement the function to meet the description.
    
the state is of type {input_schema} and the function is of type {output_schema}
The general idea is that the implementation would involve extracting the input from the state, and updating the state with the output. Description contains the logic for this blackbox
"""
    
def identify_node(state: NodeBuilderState):
    """Identify the node and return the information."""
    # Extract the information from the state
    llm_with_structured_output = llm.with_structured_output(NodeType)
    node_type_identifier_prompt = """
        You are an expert in identifying whether a function definition could be handled via simple algorithmic logic, or requires the use of large language model aka LLMs 
        
        Examples:
        Q: The node's job is to generate a plan given user query.
        A: planner
        
        Q: Node adds two numbers
        A: determinisitc
        
        Q: Node needs to analyze user query and classify sentiment
        A: planner
        
        Q: Node performs function that needs critical thinking or fuzzy logic
        A: planner
        """
    type_of_node = llm_with_structured_output.invoke([SystemMessage(content=node_type_identifier_prompt)]+ [HumanMessage(content = node_info_prompt.format(
        schema_info=state["schema_info"],
        input_schema=state["input_schema"],
        output_schema=state["output_schema"],
        description=state["description"],
        function_name=state["function_name"]))])
    
    return {"node_type": type_of_node.node_type, "messages": [HumanMessage(content=node_info_prompt.format(
        schema_info=state["schema_info"],
        input_schema=state["input_schema"],
        output_schema=state["output_schema"],
        description=state["description"],
        function_name=state["function_name"])) ], 
            "node_info": node_info_prompt.format(
        schema_info=state["schema_info"],
        input_schema=state["input_schema"],
        output_schema=state["output_schema"],
        description=state["description"],
        function_name=state["function_name"])}

def deterministic_code_gen(state: NodeBuilderState):
    """Generate the code for the node."""
    prompt = ChatPromptTemplate.from_template(
        """Generate the python code for the function {function_name}.
You are provided with the following information about the node:
The schema information is as follows: {schema_info}
The input schema is: {input_schema}
The output schema is: {output_schema}
The description of the function is: {description}

Implement the function to meet the requirements.
""")
    code = llm.invoke(prompt.format(
        schema_info=state["schema_info"],
        input_schema=state["input_schema"],
        output_schema=state["output_schema"],
        description=state["description"],
        function_name=state["function_name"]))
    return {"final_code": code.content}


# Planner

In [ ]:
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub
from langchain_openai import ChatOpenAI
from typing import Union
from langgraph.prebuilt import create_react_agent

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4o-mini")


class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )
    justification: str = Field(description = "justification for the proposed plan")

planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """For the given node information by user, come up with a simple step by step plan to implement the node.
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps.

You need to closely adhere to one of the following node building strategies
<NodeBuildingStrategies>


*** prompting + tool binding ***

When to use:
a. if the node's function is to answer a user_query which needs factual information to answer
b. personal assistant which may need to respond with which tool should be used if any to meet the requirements 
c. Whenever the node involves a LLM(model) needing sensors or actuators, for information retrieval, and allowing llm(model) to 'DO' stuff
d. One of the best use cases of this technique is when you know that there would be 'n' different tools which could be used in different scenarios to handle a query in a particular domain.

Algorithm:
1. Write a prompt to handle the user query
2. Use the toolset_generation capbility to pair the prompts with appropriate tools. 

IMPORTANT: this node is not responsible for execution, that will be handled separately, PLAN SHOULD NOT INCLUDE TOOL CALLING/EXECUTION
**************************************


***prompting + structured_output***
When not to use: 
DONOT USE if the user query needs to be answered based on factual data, refer to 'prompting + tool_binding' for the same.
do not rely on LLM's ability to answer factual data, always refer to tool_binding strategy
When to use:
a. Reduced Hallucinations: By enforcing adherence to a JSON Schema, structured outputs minimize the chance of models generating incorrect or irrelevant data. 
b. Simplified Prompting: You don't need overly complex or specific prompts to get consistently formatted output, as the schema provides the structure. 
c. Reliable Type-Safety: Structured outputs ensure that the model always generates data that fits the defined schema, eliminating the need for validation or retries due to format errors. 
d. Building Complex Workflows: Structured outputs are particularly useful for building multi-step workflows where the output of one step serves as input for the next. 
e. Integration with Databases and APIs: When integrating with systems that require structured data formats (like databases or APIs), structured outputs ensure consistency and avoid integration problems. 
f. Function Calling and Data Extraction: Structured outputs are recommended for function calling and extracting structured data from various sources. 
g. Consistent and Verifiable Output: The predictable format of structured outputs makes it easier to test, debug, and evaluate applications that rely on them. 
h. Explicit Refusals: You can now detect model refusals programmatically when using structured outputs, as the model will explicitly return a structured error message instead of a text-based one. 


Steps to follow:
1. Generate a prompt for the given ask
2. Use structured output functionality 

***************************************

*** Interrupt ***
Use case: 
a. Reviewing tool calls: Humans can review, edit, or approve tool calls requested by the LLM before tool execution.
b. Validating LLM outputs: Humans can review, edit, or approve content generated by the LLM.
c. Providing context: Enable the LLM to explicitly request human input for clarification or additional details or to support multi-turn conversations.

Algorithm:
1. follow the interrupt pattern
***************************************

</NodeBuildingStrategies>


Read the complete 'NodeBuildingStrategies' section to select the best strategy and create a tailored plan accordingly

Filters on plan: 
1. The result of the final step should be the final answer
2. Do not add steps like extract _user_query , tool_execution, or update state
""",


        ),
        ("placeholder", "{messages}"),
    ]
)
planner = planner_prompt | ChatOpenAI(
    model="gpt-4o-mini", temperature=0
).with_structured_output(Plan) 

plan_filter_prompt = ChatPromptTemplate.from_template("""
You have been provided with a plan
<Plan>
{original_plan}
</Plan>

You are tasked with filtering this prompt
1. Do not focus on input manipulation - any plan steps like extracting input/decomposing etc.
2. Do not focus on updating output to state - store ouput to final state

Check each step of the plan, and filter them to give final plan
""")

def plan_step(state: NodeBuilderState):
    plan: Plan = planner.invoke({"messages": state["messages"]})
    llm_plan = ChatOpenAI(model="gpt-4o-mini", temperature=0).with_structured_output(Plan)
    filtered_plan = llm_plan.invoke(plan_filter_prompt.format(original_plan="\n".join(plan_step for plan_step in plan.steps)))
    return {"plan": filtered_plan.steps}

    



class Response(BaseModel):
    """Response to user."""

    response: str


class Act(BaseModel):
    """Action to perform."""

    action: Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response. "
        "If you need to further use tools to get the answer, use Plan."
    )
    justification: str = Field(description= "Justificsation for choosing this action")


replanner_prompt = ChatPromptTemplate.from_template(
    """
    
You are tasked with taking next action based on the current state of the workflow.
You have two actions to choose from: 'Response' or 'Plan'
You will be provided an 'OriginalPlan' and 'PastSteps' of the plan that have been executed below:
<OriginalPlan>
{plan}
</OriginalPlan>

<PastSteps>
{past_steps}
</PastSteps>


If the 'PastSteps' section covers all the components in the 'OriginalPlan', then no more steps are needed, respond back to the user using Response action.

Otherwise, fill out the plan with the 'Plan' action. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan.
"""
)


replanner = replanner_prompt | ChatOpenAI(
    model="gpt-4o-mini", temperature=0
).with_structured_output(Act)

def replan_step(state: NodeBuilderState):
    output = replanner.invoke(state)
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        return {"plan": output.action.steps}


def should_end(state: NodeBuilderState
):
    if "response" in state and state["response"]:
        return "code_compiler"
    else:
        return "ai_node_gen_supervisor"
    

# Supervisor creation function

In [436]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from typing import  Literal, TypedDict
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import SystemMessage, AIMessage

def make_supervisor_node(llm: BaseChatModel, members: list[str]) -> str:
    options = ["FINISH"] + members


    class Router(BaseModel):
        """Worker to route to next. If no workers needed, route to FINISH."""
        next: Literal[*options] = Field(description="worker to act next") # type: ignore
        reason: str = Field(description="justification for selecting the particular next worker and expectations from it.")


    def supervisor_node(state: NodeBuilderState) -> Command[Literal[*members, "__end__"]]:
        """An LLM-based router."""
        plan = state["plan"]
        plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
        task = plan[0]
        task_formatted = f"""For the following plan:
{plan_str}\n\nYou are tasked with executing step {1}, {task}.

You are a supervisor tasked with managing a conversation between the
        following workers: {members}. Given the step,
        respond with the worker to act next. The worker will perform a
        task and respond with their results and status. When finished,
        respond with FINISH.
        
        As a supervisor you need to identify which worker to call to execute the step along with justification.
"""

        messages =  [("user", task_formatted)]+[HumanMessage(content=state["node_info"])]
        response = llm.with_structured_output(Router).invoke(messages)
        goto = response.next
        if goto == "FINISH":
            return Command(goto="replan", update={"next": "replan",
                                          "past_steps": [(task, response.reason)],
                                          "messages": [AIMessage(content= response.reason)]})
               
        return Command(goto=goto, update={"next": goto,
                                          "task": task,
                                          "messages": [AIMessage(content= response.reason)]})

    return supervisor_node


# TOOLSETGEN agent

In [437]:
from langchain_core.messages import HumanMessage


toolset_retrieval_prompt = """
    User will provide information about a node, Your task is to see if meeting the requirements of the node needs llm(aka model)'s tool-binding capability.
    
    Information about tool calling capabilities of LLMs, how to create tools and how to bind them with a LLM is provided below:

<NEED_FOR_TOOL_BINDING>
a. if the node's function is to answer a user_query which needs factual information to answer
b. personal assistant which may need to respond with which tool should be used if any to meet the requirements 
c. Whenever the node involves a LLM(model) needing sensors or actuators, for information retrieval, and allowing llm(model) to 'DO' stuff
d. One of the best use cases of this technique is when you know that there would be 'n' different tools which could be used in different scenarios to handle a query in a particular domain.
</NEED_FOR_TOOL_BINDING>

<TOOL_BINDING_EXAMPLE>
Example 1: 
``` python
from typing import Literal
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.messages import AIMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

# Define the tools needed by the LLM

@tool
def get_weather(location: str):
    \"\"\"Call to get the current weather.\"\"\"
    if location.lower() in ["sf", "san francisco"]:
        return "It's 60 degrees and foggy."
    else:
        return "It's 90 degrees and sunny."


@tool
def get_coolest_cities():
    \"\"\"Get a list of coolest cities\"\"\"
    return "nyc, sf"

tools = [get_weather, get_coolest_cities]

# Bind the model(llm) with tools
model_with_tools = ChatAnthropic(
    model="claude-3-haiku-20240307", temperature=0
).bind_tools(tools)

# Generate a tool node.
tool_node = ToolNode(tools)

# conditional edge
def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

# node implementation
def call_model(state: MessagesState):
    \"\"\"This node answers questions related to weather using a varied weather related toolset\"\"\"
    messages = state["messages"]
    response = model_with_tools.invoke([SystemMessage(content= "Please analyze the following weather-related query and provide a detailed response with relevant information: You have tools like get_weather and get_coolest_cities to help you answer the queries" )] + messages)
    return {"messages": [response]}


# Stategraph compilation
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, ["tools", END])
workflow.add_edge("tools", "agent")
app = workflow.compile()
```
</TOOL_BINDING_EXAMPLE>

<Output> 
Identify if tool_binding is relevant based on information 'NEED_FOR_TOOL_BINDING'
if yes: identify what kind of tools may be needed to be used by a LLM, and then write code for the llm binding functionality, use 'TOOL_BINDING_EXAMPLE'
if no: just respond no tool_binding needed.

Only write python code as output
</Output>
    """

def toolset_generation(state: NodeBuilderState) -> Command[Literal["ai_node_gen_supervisor"]]:
    """Generate the code for the node."""
    result = llm.invoke([SystemMessage(content=toolset_retrieval_prompt)] + state["messages"])
          
    return Command(
        update={
            "past_steps": [(state["task"], result.content)],
            "messages": [
                HumanMessage(content=result.content, name="toolset_generator")
            ]
        },
        goto="replan",
    )

# PROMPTGEN agent

In [438]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate


prompt_vector_store = get_vector_store(embeddings, "promptguide")
prompt_retriever = prompt_vector_store.as_retriever()
prompt_gen_retriever = create_retriever_tool(prompt_retriever, "Retrieve_info_on_prompting", "Search information about what are different prompting techniques relevant to the user requirements.")

promptgen_prompt = ChatPromptTemplate.from_messages([
         ("system", """
    You are a ReAct (Reasoning and Act) agent.
    You are tasked with generating a prompt to meet the objectives of the langgraph node.
    The langgraph node information is provided. 
    
    For example: 
    User query: the node is supposed to generate a plan using llms
    Thought: I need to generate a prompt that will make the LLM generate a plan for the given task.
    Action: Use the  'Retrieve_info_on_prompting' tool to search for plan-and-execute prompting techniques.
    Observation: I found a plan-and-execute prompting technique that can generate a plan for the given task.
    Action: I will customize the observed prompt to meet the requirements of the node.
    
    IMPORTANT: Your final output will be only a prompt, no code
    """),
    ("placeholder", "{messages}"),
    ])
prompt_gen_agent = create_react_agent(llm, tools=[prompt_gen_retriever], prompt = promptgen_prompt)

def prompt_generation(state: NodeBuilderState) -> Command[Literal["ai_node_gen_supervisor"]]:
    """Generate the code for the node."""
    response = prompt_gen_agent.invoke({"messages": state["messages"]})
    return Command(
        update={
            "past_steps": [(state["task"], response["messages"][-1].content)],
            "messages": [
                HumanMessage(content=response["messages"][-1].content, name="prompt_generator")
            ]
        },
        goto="replan",
    )


# STRUCTUTRED OUTPUT GEN

In [439]:


struc_output_prompt = """
    User will provide you with the information about the node,  you are supposed to analyze the information and see if it requires LLM (aka model)'s 'with_structured_output()' function.

Here is information about structured outputs, why and when to use structured outputs:
<AboutStructuredOutput>
Structured output is beneficial in situations where consistent and verifiable data formats are needed, especially when integrating with databases, APIs, or building complex workflows. It helps reduce hallucinations, simplifies prompting, and enables reliable type-safety, making applications more predictable and easier to evaluate. 

Here's a more detailed breakdown:
1. Reduced Hallucinations: By enforcing adherence to a JSON Schema, structured outputs minimize the chance of models generating incorrect or irrelevant data. 
2. Simplified Prompting: You don't need overly complex or specific prompts to get consistently formatted output, as the schema provides the structure. 
3. Reliable Type-Safety: Structured outputs ensure that the model always generates data that fits the defined schema, eliminating the need for validation or retries due to format errors. 
4. Building Complex Workflows: Structured outputs are particularly useful for building multi-step workflows where the output of one step serves as input for the next. 
5. Integration with Databases and APIs: When integrating with systems that require structured data formats (like databases or APIs), structured outputs ensure consistency and avoid integration problems. 
6. Function Calling and Data Extraction: Structured outputs are recommended for function calling and extracting structured data from various sources. 
7. Consistent and Verifiable Output: The predictable format of structured outputs makes it easier to test, debug, and evaluate applications that rely on them. 
8. Explicit Refusals: You can now detect model refusals programmatically when using structured outputs, as the model will explicitly return a structured error message instead of a text-based one. 

In essence, structured output provides a robust and predictable way to manage the output of language models, making them more reliable and easier to integrate into various applications. 
</AboutStructuredOutput>

Here is an example showing how to use 'with_structured_output' method is below:
<StructuredOutputExample>
Here is an example of how to use structured output. In this example, we want the LLM to generate and fill up the pydantic class Joke, based on user query. 
``` python
from typing import Optional
from pydantic import BaseModel, Field


# Pydantic class for structured output
class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )

class JokeBuilderState(MessagesState):
    joke: Joke = Field(description= "joke generated by the GenerateJoke node.")

def GenerateJoke(state: JokeBuilderState):
    structured_llm = llm.with_structured_output(Joke)
    joke: Joke = structured_llm.invoke("Tell me a joke about cats")
    return { "joke": joke }
```

Output:
Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!', rating=7)
</StructuredOutputExample>
    
<Notes> 
1. The structured output pydantic class is not to be the same as the input_schema or output_schema, need to be specific
2. Do not hallucinate or write your own code to implement structured output, refer to 'StructuredOutputExample' section
</Notes>
    
<Output>
Check if the node needs llm's structured output functionality based on 'AboutStructuredOutput' section.
If yes, you are supposed to generate the code for the structured output functionality for the llm to use. 
If not needed, just mention there is no need for structured output functionality.
</Output>
    """

def structured_output_generation(state: NodeBuilderState) -> Command[Literal["ai_node_gen_supervisor"]]:
    """Generate the code for the node."""
    # fetch_documents("https://python.langchain.com/docs/how_to/structured_output/")
    result = llm.invoke( [SystemMessage(content=struc_output_prompt)]  + state["messages"])

    return Command(
        update={
            "past_steps": [(state["task"], result.content)],
            "messages": [
                HumanMessage(content=result.content, name="struct_output_generator")
            ]
            
        },
        goto="replan",
    )


# Interrupt gen

In [ ]:

interrupt_gen_prompt: str = """
    User will provide you with the information about the node, you are supposed to analyze the information and see if it requires interrupt functionality.
    
    Following is the information about the interrupt functionality, what is the purpose of interrupt, design patterns, how to implement them:
    <InterruptInfo>
    The interrupt function in LangGraph enables human-in-the-loop workflows by pausing the graph at a specific node, presenting information to a human, and resuming the graph with their input. This function is useful for tasks like approvals, edits, or collecting additional input. The interrupt function is used in conjunction with the Command object to resume the graph with a value provided by the human.


``` python
from langgraph.types import interrupt

def human_node(state: State):
    value = interrupt(
        # Any JSON serializable value to surface to the human.
        # For example, a question or a piece of text or a set of keys in the state
       {
          "text_to_revise": state["some_text"]
       }
    )
    # Update the state with the human's input or route the graph based on the input.
    return {
        "some_text": value
    }

graph = graph_builder.compile(
    checkpointer=checkpointer # Required for `interrupt` to work
)

# Run the graph until the interrupt
thread_config = {"configurable": {"thread_id": "some_id"}}
graph.invoke(some_input, config=thread_config)

# Resume the graph with the human's input
graph.invoke(Command(resume=value_from_human), config=thread_config)
```
    </InterruptInfo>
    
    <Output>
    Unless explicitly mentioned in node requirements, human-in-loop aka interrupt is not needed in the scenario.
    If needed: implement the code with interrupt functionality tailored to the use case
    If not needed: just respond that interrupt functionality is not needed
    </Output>
    """

def interrupt_generation(state: NodeBuilderState) -> Command[Literal["ai_node_gen_supervisor"]]:
    """Generate the code for the node."""
    # fetch_docs=fetch_documents("https://langchain-ai.github.io/langgraph/concepts/human_in_the_loop/")
    response = llm.invoke( [SystemMessage(content=interrupt_gen_prompt)]  + state["messages"])
    return Command(
        update={
            "past_steps": [(state["task"], response.content)],
            "messages": [
                HumanMessage(content=response.content, name="interrupt_generator")
            ]
        },
        goto="replan",
    )


# Code compiler

In [441]:

code_compiler_prompt = """
    You are tasked with furnishing a final output code for the user provided node information. 
    
    Analyze the message history, you will find multiple code pieces
    You need to carefully merge all the generate code together to furnish a final output.
    
    Only write python code with comments as output, no other explanations
    """


def code_compiler(state: NodeBuilderState):
    """Generate the code for the node."""
    response = llm.invoke([SystemMessage(content=code_compiler_prompt)]  + state["messages"])
    return {"final_code": response.content}


In [442]:
ai_node_gen_supervisor = make_supervisor_node(llm, ["prompt_generation", "toolset_generation", "structured_output_generation", "interrupt_generation"])

# Graph creation

In [443]:
from langgraph.graph.state import CompiledStateGraph
from langgraph.graph import StateGraph, START
from langgraph.checkpoint.memory import InMemorySaver


workflow = StateGraph(NodeBuilderState)
workflow.add_node("identify_node", identify_node)
workflow.add_node("deterministic_code", deterministic_code_gen)
workflow.add_node("replan", replan_step)
workflow.add_node("planner", plan_step)
workflow.add_node("ai_node_gen_supervisor", ai_node_gen_supervisor)
workflow.add_node("prompt_generation", prompt_generation)
workflow.add_node("toolset_generation", toolset_generation)
workflow.add_node("structured_output_generation", structured_output_generation)
workflow.add_node("interrupt_generation", interrupt_generation)
workflow.add_node("code_compiler", code_compiler)

workflow.add_edge(START, "identify_node")
workflow.add_conditional_edges("identify_node", determine_node_type, ["deterministic_code", "planner"])
workflow.add_edge("deterministic_code", END)
workflow.add_edge("planner", "ai_node_gen_supervisor")
workflow.add_conditional_edges(
    "replan",
    # Next, we pass in the function that will determine which node is called next.
    should_end,
    ["ai_node_gen_supervisor", "code_compiler"],
)
workflow.add_edge("code_compiler", END)

checkpointer = InMemorySaver()
node_to_code_app : CompiledStateGraph = workflow.compile(checkpointer=checkpointer)